**Instructions**

In this activity, we will look at another example. Your task is to understand the problem and write down all the steps to set up ANOVA. After the next lesson, we will ask you to solve this problem using Python. Here are the steps that you would need to work on: 
- Null hypothesis 
- Alternate hypothesis 
- Level of significance 
- Test statistic 
- P-value 
- F table

*Context* <br>
Suppose you are working as an analyst in a microprocessor chip manufacturing plant. You have been given the task of analyzing a plasma etching process with respect to changing Power (in Watts) of the plasma beam. Data was collected and provided to you to conduct statistical analysis and check if changing the power of the plasma beam has any effect on the etching rate by the machine. You will conduct ANOVA and check if there is any difference in the mean etching rate for different levels of power. You can find the data in anova_lab_data.xlsx. Data was collected randomly and provided in the table.

- State the null hypothesis and the alternate hypothesis
- What is the significance level?
- What are the degrees of freedom (DoF) of Model, Error Terms, and Total?
- Use Python to conduct ANOVA. What conclusions can you draw from the experiment and why?

In [21]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [22]:
data = pd.read_excel('anova_lab_data.xlsx')
data

,Power,Etching Rate
0,160 W,5.43
1,180 W,6.24
2,200 W,8.79
3,160 W,5.71
4,180 W,6.71
5,200 W,9.20
6,160 W,6.22
7,180 W,5.98
8,200 W,7.90
9,160 W,6.01


**Null hypothesis and alternate hypothesis**

H0: Changing the power of the plasma beam has no effect on the machine's etching rate. μ_160 = μ_180 = μ_200 <br>
HA: Changing the power of the plasma beam has an effect on the machine's etching rate. μ_160 /= μ_180 or μ_200 

BACKGROUND:<br>
Etching rate refers to the speed at which material is removed or etched from a surface during a specific etching process. It is a measure of how quickly the etching solution or plasma beam erodes or dissolves the material being etched. The process which produces the highest etching rate would be our choice.

**Significance Level** 

alpha = 0.05

**Degrees of Freedom (DoF) of Model, Error Terms, and Total**

We have 5 different observations for each power level, teherfore the degrees of freedom for the hypothesis testing are:

Total DoF = 14 <br>
DoF of Model = 2 <br>
DoF of Error Terms = 12 <br>

**ANOVA with Python. Conclusions.**

In [23]:
data.columns

Index(['Power ', 'Etching Rate'], dtype='object')

In [24]:
data = data.rename(columns={'Power ': 'Power', 'Etching Rate': 'Etching_Rate'})

In [25]:
data.groupby('Power').agg(np.mean)

,Etching_Rate
Power,
160 W,5.792
180 W,6.238
200 W,8.318


At first glance, the process with power at 200 W appears to be the best one.

In [26]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# 'C' before 'Display_design' indicates the variable should be treated as categorical rather than continuous. 
model = ols('Etching_Rate ~ C(Power)',data=data).fit()
sm.stats.anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
C(Power),2.0,18.176653,9.088327,36.878955,0.000008
Residual,12.0,2.957240,0.246437,NaN,NaN


The p-value is very small, smaller than the significane level. Therefore we reject the null hypothesis and assume that one power level is providing a different etching rate than the rest. We check which one with the t-test.

In [27]:
from scipy.stats import ttest_ind
# Filtering for rows == 200 W and column 'Etching_Rate'
display_a = data[data['Power'] == '200 W']['Etching_Rate']
# I iterate through the different unique values of Display_design
for display in data['Power'].unique():
    display_b = data[data['Power'] == display]['Etching_Rate']
    print(display, ttest_ind(display_a, display_b))

160 W Ttest_indResult(statistic=7.611403634613074, pvalue=6.237977344615716e-05)
180 W Ttest_indResult(statistic=5.827496614588661, pvalue=0.0003926796476049085)
200 W Ttest_indResult(statistic=0.0, pvalue=1.0)


**Conclusion**

When comparing the highest power with the other two we observe small p-values in both cases (rejecting the null hypothesis). Both statistic values are positive, which means that the average etching rate is higher for power at 200 Watt.